[깃허브 참고](https://github.com/jx-dohwan/KoBART_generation_summary_service/blob/main/make_models/KoBART_Summary_v3.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#pip install datasets
#pip install transformers
#pip install Rouge

SyntaxError: ignored

In [ ]:
#!pip uninstall transformers
#!pip install transformers==4.25.1

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.29.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.29.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.5 MB/s eta 0:00:00


In [ ]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import torch
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm
import glob
import json
import os

In [ ]:
transformers.__version__ # 4.25.1에 맞추기

'4.25.1'

### dir 변경 필요(1)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train.csv')
valid = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid.csv')

In [ ]:
train.head()

,Text,Summary,Topic
0,웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 ...,20년 동안 감자탕 집 주차도 했었는데 감자탕 집이 장사 잘 되니까 배 아파서 주차...,주거와생활
1,아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는...,제주시에 쿠팡 로켓 배송이나 택배가 오는지 물어보고 있다.,주거와생활
2,#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면...,이불 위에 머리 망이 없다고 하니까 그럼 넣은 것 같다고 했다.,주거와생활
3,분양가 얼마여써? 여기 일억육천구백이더라ㅔ ㅋㅋㅋㅋㅋ걍 여기서 살고싶다..........,분양가 이억이 넘는데 일억은 대출로 받아야 한다.,주거와생활
4,지금 뭔가 카톡 느리지않녀 새해라 다들 톡하느라 터진건가 ㅇㅇ 약간 버벅댐 ㅇㅇㅇ ...,새해에 다들 카톡을 해서 그런지 카톡이 약간 느렸지만 괜찮아졌다.,주거와생활


In [ ]:
train['Topic'].unique()

array(['주거와생활', '시사교육', '상거래(쇼핑)', '개인및관계',
       '행사', '미용과건강', '일과직업', '식음료', '여가생활'],
      dtype=object)

### 전처리

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [ ]:
train_texts = data_process(train['Text'])
val_texts = data_process(valid['Text'])

train_df = pd.DataFrame(zip(train_texts,train['Summary']), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_texts,valid['Summary']), columns=['Text', 'Summary'])

100%|██████████| 279992/279992 [00:09<00:00, 29118.43it/s]


In [ ]:
train_df.head()

,Text,Summary
0,웃긴게 20년동안 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 그 가...,20년 동안 감자탕 집 주차도 했었는데 감자탕 집이 장사 잘 되니까 배 아파서 주차...
1,아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는...,제주시에 쿠팡 로켓 배송이나 택배가 오는지 물어보고 있다.
2,#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 읭 만약에 잇으면 침대 ...,이불 위에 머리 망이 없다고 하니까 그럼 넣은 것 같다고 했다.
3,분양가 얼마여써 여기 일억육천구백이더라 걍 여기서 살고싶다 나는 이억 넘어써 와 근...,분양가 이억이 넘는데 일억은 대출로 받아야 한다.
4,지금 뭔가 카톡 느리지않녀 새해라 다들 톡하느라 터진건가 약간 버벅댐 근데 나는 아...,새해에 다들 카톡을 해서 그런지 카톡이 약간 느렸지만 괜찮아졌다.


In [ ]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

print(train_data)
print(val_data)
print(test_samples)

Dataset({
    features: ['Text', 'Summary'],
    num_rows: 279992
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 35004
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 35004
})


In [ ]:
#model_checkpoints = "/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500"

model_checkpoints = 'gogamza/kobart-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

# special_words = [
#                 "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
#                 "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
#                 "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
#                 "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
#                 "#개인 및 관계#", "#미용과 건강#", "#상거래(쇼핑)#", "#시사/교육#", "#식음료#", 
#                 "#여가 생활#", "#일과 직업#", "#주거와 생활#", "#행사#","[sep]"
#                 ]

# tokenizer.add_special_tokens({"additional_special_tokens": special_words})
# model.resize_token_embeddings(len(tokenizer))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
# t_len = [len(tokenizer.encode(s)) for s in tqdm(train_df['Text'])]
# s_len = [len(tokenizer.encode(s)) for s in tqdm(train_df['Summary'])]

# fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
# axes[0].hist(t_len, bins=50, color="C0", edgecolor="C0")
# axes[0].set_title("Dialogue Token Length")
# axes[0].set_xlabel("Length")
# axes[0].set_ylabel("Count")
# axes[1].hist(s_len, bins=50, color="C0", edgecolor="C0")
# axes[1].set_title("Summary Token Length")
# axes[1].set_xlabel("Length")
# plt.tight_layout()
# plt.show()

In [ ]:
max_input = 256
max_target = 64
ignore_index = -100# tokenizer.pad_token_id

def add_ignored_data(inputs, max_len, ignore_index):
  if len(inputs) < max_len:
      pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
      inputs = np.concatenate([inputs, pad])
  else:
      inputs = inputs[:max_len]

  return inputs

def add_padding_data(inputs, max_len):
    pad_index = tokenizer.pad_token_id
    if len(inputs) < max_len:
        pad = [pad_index] *(max_len - len(inputs))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs 

def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []
    bos = tokenizer('')['input_ids']
    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)   
        dec_input_id = bos
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

Map:   0%|          | 0/279992 [00:00<?, ? examples/s]

Map:   0%|          | 0/35004 [00:00<?, ? examples/s]

In [ ]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)  

### dir 변경 필요 (2)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    ##output_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint2/KoBART_Summary_v3",
    output_dir='/content/drive/MyDrive/BOAZ_miniProject2/results',
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=128,  # demo
    per_device_eval_batch_size=256,
    learning_rate=3e-05,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    ##logging_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/logs2",
    logging_dir='/content/drive/MyDrive/BOAZ_miniProject2/logs',
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch',
)

### 여기서 오류나면 아래 방법 참고

```
pip uninstall transformers
pip install transformers==4.25.1
```
- transformers 버전 바꿔서 다시 설치하고, 런타임 다시 시작하면 됨  
- 찾아보니깐 4.28.0버전으로 해도 될 것 같긴 한데, 깃허브에 4.25.1버전으로 했다고 써있어서 버전 맞춰줬음! 


In [ ]:
#import torch_optimizer as optim
#from transformers import AdamW
#optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?

In [ ]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
#import warnings
#warnings.filterwarnings('ignore')

gpu 메모리 부족 시


```
import torch, gc
torch.cuda.empty_cache()
gc.collect()
```



In [ ]:
# import torch, gc
# torch.cuda.empty_cache()
# gc.collect()

### 여기서 gpu 메모리 부족해서 이후로 실행 못함!!
- 그리고 cpu로 돌렸을 때 `FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version.` 이 메시지 뜨고 실행이 중단되었음!
- warning 찾아봤는데.. 안 나오도록 해결은 못함(근데 깃허브에도 같은 warning이 있긴함)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 279992
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 10940
  Number of trainable parameters = 123884544


OutOfMemoryError: ignored

In [ ]:
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?

# trainer = Seq2SeqTrainer(
#     model, 
#     training_args,
#     train_dataset=train_tokenize_data,
#     eval_dataset=val_tokenize_data,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
# )

# trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
def generate_summary(test_samples, model):

    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)

    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=5, no_repeat_ngram_size=3,
                            attention_mask=attention_mask, 
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            eos_token_id=tokenizer.eos_token_id,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

summaries_after_tuning=[]
for test_sample in tqdm(test_samples):
    summaries_after_tuning.append(generate_summary(test_sample, model)[1])
summaries_after_tuning = list(itertools.chain(*summaries_after_tuning))

  1%|          | 291/35004 [17:24<34:36:22,  3.59s/it]
Exception ignored in: <function tqdm.__del__ at 0x7f40d47ceb00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1144, in __del__
    def __del__(self):
KeyboardInterrupt: 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-19-bdbababca42a>:23 in <cell line: 22>                                            │
│ <ipython-input-19-bdbababca42a>:13 in generate_summary                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1608 in generate        │
│                                                                                                  │
│   1605 │   │   │   │   **model_kwargs,                                                           │
│   1606 │   │   │   )                                                                             │
│   1607 │   │   │   # 12. run beam search                                                         │
│ ❱ 1608 │   │   │   return self.beam_search(                                                      │
│   1609 │   │   │   │   input_ids,                                                                │
│   1610 │   │   │   │   beam_scorer,                                                              │
│   1611 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2799 in beam_search     │
│                                                                                                  │
│   2796 │   │   │                                                                                 │
│   2797 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2798 │   │   │                                                                                 │
│ ❱ 2799 │   │   │   outputs = self(                                                               │
│   2800 │   │   │   │   **model_inputs,                                                           │
│   2801 │   │   │   │   return_dict=True,                                                         │
│   2802 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [ ]:
rouge.get_scores(summaries_after_tuning, test_samples["Summary"], avg=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-20-6cf6594244b3>:1 in <cell line: 1>                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/rouge/rouge.py:104 in get_scores                         │
│                                                                                                  │
│   101 │   │   │   hyps, refs = zip(*hyps_and_refs)                                               │
│   102 │   │                                                                                      │
│   103 │   │   assert(isinstance(hyps, type(refs)))                                               │
│ ❱ 104 │   │   assert(len(hyps) == len(refs))                                                     │
│   105 │   │                                                                                      │
│   106 │   │   if not avg:                                                                        │
│   107 │   │   │   return self._get_scores(hyps, refs)                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError

In [ ]:
for i in range(0, len(summaries_after_tuning), 1000):
    print('idx_{} '.format(i))
    print("Summary after \n"+ summaries_after_tuning[i])
    print("")
    print("Target summary \n"+ test_samples["Summary"][i])
    print("")
    print('Text'+ test_samples["Text"][i])
    print('-'*100)
    print("") 

idx_0 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-21-407cd1591b87>:3 in <cell line: 1>                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: can only concatenate str (not "list") to str